# AUS vs IND (CWC 2023) match predictions
![](https://assets-in.bmscdn.com/nmcms/events/banner/desktop/media-desktop-india-vs-australia-icc-mens-cwc-2023-0-2023-8-24-t-5-48-10.jpg)

## Load data

In [26]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
# Load data into csv files
train = pd.read_csv("C:/Users/relat/Downloads/data_zpyYWs0.csv")
sub = pd.read_csv("C:/Users/relat/Downloads/sample_submission_Gk1Mne1.csv")

In [32]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2575 entries, 0 to 2574
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   player_id      2575 non-null   int64 
 1   player_name    2575 non-null   object
 2   runs_scored    2575 non-null   object
 3   wickets        2575 non-null   object
 4   runs_conceded  2575 non-null   object
 5   catches        2575 non-null   object
 6   stumpings      2575 non-null   object
 7   match_date     2575 non-null   object
 8   opposition     2575 non-null   object
 9   match_id       2575 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 201.3+ KB


## Preprocessing

In [33]:
# make not out column
# train['not_out'] = train['runs_scored'].apply(lambda x: True if x[-1]=='*' else False)
# clean runs scored and convert to int
train['runs_scored'] = train['runs_scored'].str.replace('*', '', regex=False)
train['runs_scored'].replace({'DNB': '0', 'TDNB': '0'}, inplace=True)
train['runs_scored'] = train['runs_scored'].astype('int')
# clean wickets and convert to int
train['wickets'] = train['wickets'].str.replace('-', '0', regex=False).astype('int')
# clean runs conceded and convert to int
# train['runs_conceded'] = train['runs_conceded'].str.replace('-', '-1', regex=False).astype('int')
# clean catches and convert to int
# train['catches'] = train['catches'].str.replace('-', '-1', regex=False).astype('int')
# clean stumpings and convert to int
# train['stumpings'] = train['stumpings'].str.replace('-', '-1', regex=False).astype('int')
# convert date to datetime
train['match_date'] = pd.to_datetime(train['match_date'])
# extract different ground names
train['ground'] = train['opposition'].apply(lambda x: x.split()[-1])
# extract opponent team
train['opposition'] = train['opposition'].apply(lambda x: x.split()[1])
train['opposition'].replace({'South': 'South Africa', 
                             'New': 'New Zealand',
                             'West': 'West Indies',
                             'Sri': 'Sri Lanka',
                             'Hong': 'Hong Kong'}, inplace=True)
# Create wk column
# train['wk'] = train['player_id'].apply(lambda x: True if x in [3,4,21,29] else False)

In [34]:
train.groupby('player_id').player_name.unique()

player_id
1             [Pat Cummins]
2             [Steve Smith]
3              [Alex Carey]
4             [Josh Inglis]
5             [Sean Abbott]
6      [Marnus Labuschagne]
7           [Cameron Green]
8          [Josh Hazlewood]
9             [Travis Head]
10            [Mitch Marsh]
11          [Glenn Maxwell]
12         [Marcus Stoinis]
13           [David Warner]
14             [Adam Zampa]
15        [Mitchell Starc.]
16           [Rohit Sharma]
17          [Hardik Pandya]
18           [Shubman Gill]
19            [Virat Kohli]
20           [Shreyas Iyer]
21               [KL Rahul]
22        [Ravindra Jadeja]
23         [Shardul Thakur]
24         [Jasprit Bumrah]
25         [Mohammed Siraj]
26          [Kuldeep Yadav]
27         [Mohammed Shami]
28    [Ravichandran Ashwin]
29           [Ishan Kishan]
30       [Suryakumar Yadav]
Name: player_name, dtype: object

In [35]:
train

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id,ground
0,1,Pat Cummins,0,3,28,0,0,2011-10-19,South Africa,1,Centurion
1,2,Steve Smith,0,0,-,0,0,2011-10-19,South Africa,1,Centurion
2,10,Mitch Marsh,8,1,19,1,0,2011-10-19,South Africa,1,Centurion
3,13,David Warner,20,0,-,0,0,2011-10-19,South Africa,1,Centurion
4,1,Pat Cummins,11,1,73,0,0,2011-10-23,South Africa,2,Gqeberha
...,...,...,...,...,...,...,...,...,...,...,...
2570,22,Ravindra Jadeja,0,3,35,0,0,2009-10-28,Australia,553,Nagpur
2571,22,Ravindra Jadeja,0,2,41,0,0,2009-10-31,Australia,554,Delhi
2572,22,Ravindra Jadeja,23,0,44,1,0,2009-11-05,Australia,555,Hyderabad
2573,22,Ravindra Jadeja,57,0,36,0,0,2009-11-08,Australia,556,Guwahati


In [36]:
train.groupby('player_id').player_id.count()

player_id
1      77
2     145
3      71
4       8
5      17
6      38
7      20
8      74
9      58
10     79
11    129
12     64
13    150
14     85
15    111
16    251
17     82
18     35
19    281
20     47
21     61
22    186
23     44
24     78
25     30
26     90
27     94
28    115
29     25
30     30
Name: player_id, dtype: int64

In [54]:
runs = []
wickets = []
for i in range(1,31):
    runs.append(train[train.player_id==i].runs_scored[-5:].mean())
    wickets.append(train[train.player_id==i].wickets[-5:].mean())
print(runs)
print(wickets)

[8.0, 23.2, 24.4, 16.6, 18.0, 56.6, 14.4, 0.2, 25.6, 11.2, 28.8, 20.6, 29.0, 3.8, 18.2, 36.6, 19.2, 37.0, 47.6, 23.8, 52.8, 16.0, 23.6, 2.4, 0.0, 2.4, 5.4, 6.4, 76.6, 18.2]
[1.8, 0.0, 0.0, 0.0, 1.0, 0.0, 0.4, 1.0, 0.6, 0.0, 0.6, 0.6, 0.0, 1.8, 0.6, 0.0, 0.4, 0.0, 0.0, 0.0, 0.0, 1.8, 1.4, 2.4, 1.4, 2.0, 1.6, 1.0, 0.0, 0.0]


## Insights

In [55]:
sub['runs'] = runs
sub['wickets'] = wickets

In [56]:
sub

,player_id,runs,wickets
0,1,8.0,1.8
1,2,23.2,0.0
2,3,24.4,0.0
3,4,16.6,0.0
4,5,18.0,1.0
5,6,56.6,0.0
6,7,14.4,0.4
7,8,0.2,1.0
8,9,25.6,0.6
9,10,11.2,0.0


In [57]:
sub.to_csv("match_preds.csv", index=False)

In [11]:
sub

,player_id,runs,wickets
0,1,30,2
1,2,24,0
2,3,45,1
3,4,5,2
4,5,9,0
5,6,34,1
6,7,76,2
7,8,1,0
8,9,23,0
9,10,1,0
